In [1]:
import json
import pennylane as qml
import pennylane.numpy as np

symbols = ["H", "H", "H"]


def h3_ground_energy(bond_length):
    
    """
    Uses VQE to calculate the ground energy of the H3+ molecule with the given bond length.
    
    Args:
        - bond_length(float): The bond length of the H3+ molecule modelled as an
        equilateral triangle.
    Returns:
        - Union[float, np.tensor, np.array]: A float-like output containing the ground 
        state of the H3+ molecule with the given bond length.
    """
    
    # Create the molecule
    d = bond_length
    coordinates = np.array([[-d/2,-d/2,0],[0,d/2,0],[d/2,-d/2,0]])
    coordinates1 = np.array([[0,0,0],[d/2,d,0],[d,0,0]])

    # Get the hamiltonian and the number of qubits
    hamiltonian , num_wires = qml.qchem.molecular_hamiltonian(symbols, coordinates, charge = 1)
    hamiltonian1 , num_wires1 = qml.qchem.molecular_hamiltonian(symbols, coordinates1, charge = 1)
    # Prepare a state using fock method
    hf = qml.qchem.hf_state(electrons = 2, orbitals = num_wires)
    hf1 = qml.qchem.hf_state(electrons = 2, orbitals = num_wires)
    # Create a device
    dev = qml.device('default.qubit',wires = num_wires)

    # Preparing ground state
    @qml.qnode(dev)
    def exp_energy(state):
        qml.BasisState(np.array(state), wires = range(num_wires))
        return qml.expval(hamiltonian)
    
    print(exp_energy(hf))
    print(exp_energy(hf1))
    
    def energy_to_minimize(angles):
        qml.BasisState(hf, wires = range(num_wires))
        qml.DoubleExcitation(angles[0], wires = [0,1,2,3])
        qml.DoubleExcitation(angles[1], wires = [0,1,4,5])
        return qml.expval(hamiltonian)

    opt = qml.GradientDescentOptimizer(stepsize = 0.1)

    angles = np.array([0.,0.], requires_grad = True)
    energies = [energy_to_minimize(angles)]
    angles_list = [angles]
    num_it = 20

    for i in range(num_it):
        angles, _ = opt.step_and_cost(energy_to_minimize,angles)
        energies.append(energy_to_minimize(angles))
        angles_list.append(angles)
        if i % 5 == 0:
            print(f"Step {i}, Energy {energies[-1]}")
    return energies[-1]

if __name__ == '__main__':
    bond_length = 0.8
    print(h3_ground_energy(bond_length))

-0.5333727980800229
-0.5333727980800229


/home/martinet/PycharmProjects/Hybrid_QML/.venv/lib/python3.10/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


TypeError: Can't find vector space for value expval(  (-0.35125712533244347) [Z4]
+ (-0.35125712533244347) [Z5]
+ (-0.28858636819703165) [Z3]
+ (-0.2885863681970316) [Z2]
+ (0.3834080218388176) [Z0]
+ (0.3834080218388176) [Z1]
+ (1.5846456169939072) [I0]
+ (-0.015359755135802166) [Y1 Y3]
+ (-0.015359755135802166) [X1 X3]
+ (0.002458307679281513) [Y0 Y2]
+ (0.002458307679281513) [X0 X2]
+ (0.13568042543550354) [Z0 Z2]
+ (0.13568042543550354) [Z1 Z3]
+ (0.1379542348931757) [Z0 Z4]
+ (0.1379542348931757) [Z1 Z5]
+ (0.147121439083462) [Z2 Z4]
+ (0.147121439083462) [Z3 Z5]
+ (0.1601747840245841) [Z2 Z5]
+ (0.1601747840245841) [Z3 Z4]
+ (0.1728047035878498) [Z0 Z3]
+ (0.1728047035878498) [Z1 Z2]
+ (0.17501280813960657) [Z0 Z5]
+ (0.17501280813960657) [Z1 Z4]
+ (0.17832025863467402) [Z0 Z1]
+ (0.1883086911405768) [Z2 Z3]
+ (0.19016756378358313) [Z4 Z5]
+ (-0.001889591922444053) [Y0 Z1 Y2]
+ (-0.001889591922444053) [X0 Z1 X2]
+ (-0.001889591922444053) [Y1 Z2 Y3]
+ (-0.001889591922444053) [X1 Z